<a href="https://colab.research.google.com/github/Jamess200/MyBirdNetTest/blob/main/HistogramAndAccuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Define path in Google Drive where you want to clone the repository
repo_path = '/content/drive/MyDrive/'

# Check if directory already exists
if not os.path.exists(repo_path):
    os.makedirs(repo_path)

# Change working directory to defined path
os.chdir(repo_path)

# Clone the repository if doesnt exist, otherwise pull latest changes
if not os.path.exists(os.path.join(repo_path, 'MyBirdNetTest')):
    !git clone https://github.com/Jamess200/MyBirdNetTest.git
else:
    os.chdir('MyBirdNetTest')
    !git pull

# Verify cloned repository
!ls

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 246.60 KiB | 222.00 KiB/s, done.
From https://github.com/Jamess200/MyBirdNetTest
   9fa538e..a8fbb7a  main       -> origin/main
Updating e310bf8..a8fbb7a
error: The following untracked working tree files would be overwritten by merge:
	data/CSV_data/BirdNET_results7.csv
	data/CSV_data/BirdNET_results8.csv
	data/CSV_data/combined.csv
Please move or remove them before you merge.
Aborting
data			 img		       json_to_csv_converter.ipynb  Time_Test_BN_Test.ipynb
Density_plot_code.ipynb  Inital_BN_Test.ipynb  README.md


In [3]:
cd /content/drive/MyDrive/MyBirdNetTest/data/CSV_data

/content/drive/MyDrive/MyBirdNetTest/data/CSV_data


In [4]:
import pandas as pd
import plotly.graph_objects as go

# Read the combined CSV file
data = pd.read_csv('combined.csv')

In [5]:
# Calculate species counts and filter those with more than 20 detections
species_counts = data['common_name'].value_counts().reset_index()
species_counts.columns = ['common_name', 'count']
filtered_species_counts = species_counts[species_counts['count'] > 20]

# Get the top 3 and bottom 3 species with more than 20 detections
top_3_species = filtered_species_counts.nlargest(3, 'count')['common_name']
bottom_3_species = filtered_species_counts.nsmallest(3, 'count')['common_name']

# Filter the data for the selected species
top_3_data = data[data['common_name'].isin(top_3_species)]
bottom_3_data = data[data['common_name'].isin(bottom_3_species)]

In [6]:
# Create histograms for top 3 species
fig_top_3 = go.Figure()

for species in top_3_species:
    fig_top_3.add_trace(go.Histogram(
        x=top_3_data[top_3_data['common_name'] == species]['common_name'],
        name=species,
        text=species,
        hoverinfo='x+y+text',
        marker=dict(color='blue', line=dict(color='black', width=1)),
        opacity=0.75
    ))

fig_top_3.update_layout(
    title='Histogram of Top 3 Species by Occurrence',
    xaxis_title="Species",
    yaxis_title="Number of Detections",
    height=400,
    width=800,
    bargap=0.1,
    barmode='overlay'
)

fig_top_3.show()

In [7]:
# Create histograms for bottom 3 species
fig_bottom_3 = go.Figure()

for species in bottom_3_species:
    fig_bottom_3.add_trace(go.Histogram(
        x=bottom_3_data[bottom_3_data['common_name'] == species]['common_name'],
        name=species,
        text=species,
        hoverinfo='x+y+text',
        marker=dict(color='blue', line=dict(color='black', width=1)),
        opacity=0.75
    ))

fig_bottom_3.update_layout(
    title='Histogram of Bottom 3 Species by Occurrence (Over 20 Detections)',
    xaxis_title="Species",
    yaxis_title="Number of Detections",
    height=400,
    width=800,
    bargap=0.1,
    barmode='overlay'
)

fig_bottom_3.show()

In [10]:
# Combine the top 3 and bottom 3 data
combined_data = pd.concat([top_3_data, bottom_3_data])

# Calculate the average confidence for the top 3 and bottom 3 species
average_confidence = combined_data.groupby('common_name')['confidence'].mean().reset_index()
average_confidence.columns = ['common_name', 'average_confidence']

# Calculate the number of detections for the top 3 and bottom 3 species
detection_counts = combined_data['common_name'].value_counts().reset_index()
detection_counts.columns = ['common_name', 'count']

# Merge the average confidence and detection counts
correlation_data = pd.merge(detection_counts, average_confidence, on='common_name')

# Calculate the correlation
correlation = correlation_data['count'].corr(correlation_data['average_confidence'])
print(f"Correlation between number of detections and average confidence: {correlation}")

# Plot the data for visualization
fig_cor = go.Figure()

fig_cor.add_trace(go.Scatter(
    x=correlation_data['count'],
    y=correlation_data['average_confidence'],
    mode='markers+text',
    text=correlation_data['common_name'],
    textposition='top center',
    marker=dict(size=10, color='lightblue', line=dict(color='black', width=1)),
))

fig_cor.update_layout(
    title='Correlation between Number of Detections and Average Confidence',
    xaxis_title='Number of Detections',
    yaxis_title='Average Confidence',
    height=600
)

fig_cor.show()

Correlation between number of detections and average confidence: 0.9717941474642028


In [28]:
import plotly.express as px
from scipy.stats import f_oneway

# Combine the top 3 and bottom 3 data
combined_data = pd.concat([top_3_data, bottom_3_data])

# Perform ANOVA
anova_result = f_oneway(
    combined_data[combined_data['common_name'] == top_3_species.iloc[0]]['confidence'],
    combined_data[combined_data['common_name'] == top_3_species.iloc[1]]['confidence'],
    combined_data[combined_data['common_name'] == top_3_species.iloc[2]]['confidence'],
    combined_data[combined_data['common_name'] == bottom_3_species.iloc[0]]['confidence'],
    combined_data[combined_data['common_name'] == bottom_3_species.iloc[1]]['confidence'],
    combined_data[combined_data['common_name'] == bottom_3_species.iloc[2]]['confidence']
)

# Round ANOVA results to 3 significant figures
anova_f_statistic = round(anova_result.statistic, 3)
anova_p_value = format(anova_result.pvalue, '.4g')

# Interpret the results
interpretation = ""
if anova_result.pvalue < 0.05:
    interpretation = "This ANOVA test indicates that there are statistically significant differences in the confidence scores among the top 3 and bottom 3 species."
else:
    interpretation = "This ANOVA test indicates that there are no statistically significant differences in the confidence scores among the top 3 and bottom 3 species."

# Print the results
print(f"ANOVA F-statistic: {anova_f_statistic}, p-value: {anova_p_value}")
print(interpretation)

# Create box plot
fig_box = px.box(
    combined_data,
    x='common_name',
    y='confidence',
    title='Box Plot of Confidence Scores for Top 3 and Bottom 3 Species',
    labels={'confidence': 'Confidence Score', 'common_name': 'Species'},
    color_discrete_sequence=['blue'],
    points=False,  # Do not show individual data points
    height=600
)

fig_box.update_layout(
    xaxis_title="Species",
    yaxis_title="Confidence Score"
)

fig_box.show()

ANOVA F-statistic: 285.153, p-value: 2.625e-298
This ANOVA test indicates that there are statistically significant differences in the confidence scores among the top 3 and bottom 3 species.
